In [2]:
import sagemaker

role_arn = sagemaker.get_execution_role()
print(role_arn)



In [1]:
# Set variables
s3_bucket='bucket'
s3_prefix='prefix'
pipeline_name='MyPipeline-2023-07-03-21-07-05'
region='eu-west-2'

# Get the pipeline ARN
pipeline_arn=$(aws sagemaker list-pipelines --pipeline-name-prefix $pipeline_name --region $region --query 'PipelineSummaries[0].PipelineArn' --output text)

# Create the rule
rule_arn=$(aws events put-rule --name 'cmg_sagemaker_pipeline' --event-pattern "{\"source\": [\"aws.s3\"], \"detail-type\": [\"ObjectCreated:*\"], \"detail\": {\"bucket\": {\"name\": [\"$s3_bucket\"]}, \"object\": {\"key\": [{\"prefix\": \"$s3_prefix\"}]}}}" --state 'ENABLED' --description 'Rule to trigger SageMaker pipeline when new file is added to S3 bucket' --region $region --query 'RuleArn' --output text)

# Add the target
aws events put-targets --rule 'my-sagemaker-pipeline-trigger-rule' --targets "Id"="1","Arn"="$pipeline_arn","Input"='{"PipelineExecutionDescription": "Started by EventBridge rule"}',"InputPath"="$.detail" --region $region

echo "Rule ARN: $rule_arn"
